# Machine Learning with Spark

Machine learning is a process for extracting patterns from your data, using statistics, linear algebra, and numerical optimization. This notebook focuses on supervised learning, with linear regression. Specifically, focuses on univariate linear regression. Univariate linear regression focuses on determining relationship between one independent (explanatory variable) variable and one dependent variable.

### Dataset

In [0]:
# the directory of the dataset, the files, etc.
display(dbutils.fs.ls('/databricks-datasets/learning-spark-v2/sf-airbnb/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/README-sf-airbnb.md,README-sf-airbnb.md,1064,1575931327000
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/lr-pipeline-model/,lr-pipeline-model/,0,1665977670565
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-clean-100p.parquet/,sf-airbnb-clean-100p.parquet/,0,1665977670565
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-clean.parquet/,sf-airbnb-clean.parquet/,0,1665977670565
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv,sf-airbnb-numeric.csv,554979,1588481870000
dbfs:/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb.csv,sf-airbnb.csv,34234636,1575931330000


In [0]:
filePath = """/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-clean.parquet/"""
airbnbDF = spark.read.parquet(filePath)

In [0]:
airbnbDF.columns
airbnbDF.select("neighbourhood_cleansed", "room_type", "bedrooms", "bathrooms",
                "number_of_reviews", "price").show(5)

+----------------------+---------------+--------+---------+-----------------+-----+
neighbourhood_cleansed| room_type|bedrooms|bathrooms|number_of_reviews|price|
+----------------------+---------------+--------+---------+-----------------+-----+
 Western Addition|Entire home/apt| 1.0| 1.0| 180.0|170.0|
 Bernal Heights|Entire home/apt| 2.0| 1.0| 111.0|235.0|
 Haight Ashbury| Private room| 1.0| 4.0| 17.0| 65.0|
 Haight Ashbury| Private room| 1.0| 4.0| 8.0| 65.0|
 Western Addition|Entire home/apt| 2.0| 1.5| 27.0|785.0|
+----------------------+---------------+--------+---------+-----------------+-----+
only showing top 5 rows

### Creating Training and Test Datasets

Before we begin feature engineering and modeling, we will divide our data set into two groups: train and test. Depending on the size of your data set, your train/test ratio may vary, but many data scientists use 80/20 as a standard train/test split.

In [0]:
trainDF, testDF = airbnbDF.randomSplit([.8, .2], seed=42)
print(f"train: {trainDF.count()} - test {testDF.count()}")

train: 5780 - test 1366

In [0]:
# caching the training set is helpful since the dataset will be used many times during eda, ml, etc.
trainDF.cache()
# this action is necessary to eagerly trigger the caching
trainDF.count()

Out[5]: 5780

In [0]:
# notice the run of this cell is less than the above, since the dataframe is cached already
trainDF.count()

Out[6]: 5780

### Preparing Features with Transformers

This step prepares the data to build a linear regression model predicting price given the number of bedrooms. Linear regression requires that all the input features are contained within a single vector in your DataFrame. Thus, we need to transform our data.

[VectorAssembler](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.ml.feature.VectorAssembler.html#vectorassembler) takes a list of input columns and creates a new DataFrame with an additional column, which we will call features. It combines the values of those input columns into a single vector.

In [0]:
from pyspark.ml.feature import VectorAssembler

vecAssember = VectorAssembler(inputCols=['bedrooms'], outputCol='features')

In [0]:
vecTrainDF = vecAssember.transform(trainDF)
vecTrainDF.select(['bedrooms', 'features', 'price']).show(3)

+--------+--------+-----+
bedrooms|features|price|
+--------+--------+-----+
 1.0| [1.0]|200.0|
 1.0| [1.0]|130.0|
 1.0| [1.0]| 95.0|
+--------+--------+-----+
only showing top 3 rows

In [0]:
trainDF.count(), len(trainDF.columns), ' ', vecTrainDF.count(), len(vecTrainDF.columns) 

Out[9]: (5780, 34, ' ', 5780, 35)

### Using Estimator to Build the Model

with `VectorAssembler`, we have our data prepared and transformed into a format that our linear regression model expects. In Spark, [LinearRegression](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html?highlight=linearregression#linearregression) is a type of estimator — it takes in a DataFrame and returns a Model. 

Estimators **learn** parameters from your data, have an estimator_name.fit() method, and are eagerly evaluated (i.e., kick off Spark jobs), whereas transformers are lazily evaluated.

In [0]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features', labelCol='price')
lrModel = lr.fit(vecTrainDF)

In [0]:
print("price = {m:.2f} * bedrooms + {b:.2f}".format(
    m=lrModel.coefficients[0], b=lrModel.intercept))

price = 123.68 * bedrooms + 47.51

### Creating a Pipeline

If we want to apply our model to our test set, then we need to prepare that data in the same way as the training set (i.e., pass it through the vector assembler). Oftentimes data preparation pipelines will have multiple steps, and it becomes cumbersome to remember not only which steps to apply, but also the ordering of the steps. 

In Spark, Pipelines are esti‐ mators, whereas PipelineModels—fitted Pipelines—are transformers.

In [0]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssember, lr])
pipelineModel = pipeline.fit(trainDF)

In [0]:
predDF = pipelineModel.transform(testDF)
predDF.select('bedrooms', 'features', 'price', 'prediction').show(10)

+--------+--------+------+------------------+
bedrooms|features| price| prediction|
+--------+--------+------+------------------+
 1.0| [1.0]| 85.0|171.18598011578285|
 1.0| [1.0]| 45.0|171.18598011578285|
 1.0| [1.0]| 70.0|171.18598011578285|
 1.0| [1.0]| 128.0|171.18598011578285|
 1.0| [1.0]| 159.0|171.18598011578285|
 2.0| [2.0]| 250.0|294.86172649777757|
 1.0| [1.0]| 99.0|171.18598011578285|
 1.0| [1.0]| 95.0|171.18598011578285|
 1.0| [1.0]| 100.0|171.18598011578285|
 1.0| [1.0]|2010.0|171.18598011578285|
+--------+--------+------+------------------+
only showing top 10 rows